## 1. Install Dependencies

PyTorch Geometric is not pre-installed on Kaggle.

In [ ]:
# Install PyTorch Geometric
!pip install -q torch-geometric torch-scatter torch-sparse

print("✓ Dependencies installed successfully")

## 2. Import Libraries

In [ ]:
import os
import sys
import json
import shutil
import torch
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from torch_geometric.utils import k_hop_subgraph
from huggingface_hub import hf_hub_download
from tqdm import tqdm

# Check environment
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 3. Configuration

Adjust these parameters if needed.

In [ ]:
# Configuration
CONFIG = {
    'output_dir': '/kaggle/working/cora_gwm_data',
    'bert_model': 'sentence-transformers/all-MiniLM-L6-v2',
    'num_hops': 5,
    'sample_size': 5,  # Max neighbors to sample per hop
    'embedding_dim': 2048,  # Target dimension for GWM-E
    'test_size': 0.2,  # 20% test split
    'random_state': 42,  # For reproducibility
    'batch_size': 32,  # BERT encoding batch size
}

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
CONFIG['device'] = device

print("Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 4. Download Raw Cora Dataset

Download from HuggingFace Hub.

In [ ]:
print("="*70)
print(" "*20 + "STEP 1: Downloading Raw Data")
print("="*70)

repo_id = "Graph-COM/Text-Attributed-Graphs"
filename = "cora/processed_data.pt"
raw_dir = Path("/kaggle/temp/cora_raw")
raw_dir.mkdir(parents=True, exist_ok=True)
dest_file = raw_dir / "data.pt"

if dest_file.exists():
    print(f"✓ Raw data already exists at {dest_file}")
else:
    print(f"Downloading from {repo_id}...")
    downloaded_path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        repo_type="dataset"
    )
    
    print(f"Copying to {dest_file}...")
    shutil.copy(downloaded_path, dest_file)

# Load and verify
print("\nLoading dataset...")
data = torch.load(dest_file, weights_only=False)

print(f"\n✓ Successfully loaded Cora dataset!")
print(f"  Nodes: {data.num_nodes:,}")
print(f"  Edges: {data.edge_index.size(1):,}")
print(f"  Classes: {len(data.label_texts)}")
print(f"  Class names: {data.label_texts}")
print(f"  Has text: {hasattr(data, 'raw_texts')}")

if not hasattr(data, 'raw_texts'):
    raise ValueError("Dataset missing 'raw_texts' attribute!")

print(f"\nExample node text:")
print(f"  {data.raw_texts[0][:200]}...")

## 5. Generate BERT Embeddings

Encode all node texts using BERT.

In [ ]:
print("="*70)
print(" "*20 + "STEP 2: Generating BERT Embeddings")
print("="*70)

print(f"Loading BERT model: {CONFIG['bert_model']}")
tokenizer = AutoTokenizer.from_pretrained(CONFIG['bert_model'])
bert_model = AutoModel.from_pretrained(CONFIG['bert_model']).to(device)
bert_model.eval()

print(f"✓ Model loaded on {device}")

# Generate embeddings
all_embeddings = []
batch_size = CONFIG['batch_size']

print(f"\nEncoding {len(data.raw_texts):,} texts (batch_size={batch_size})...")
with torch.no_grad():
    for i in tqdm(range(0, len(data.raw_texts), batch_size), desc="BERT encoding"):
        batch_texts = data.raw_texts[i:i+batch_size]
        
        # Tokenize
        encoded = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        ).to(device)
        
        # Get embeddings (mean pooling)
        outputs = bert_model(**encoded)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        all_embeddings.append(embeddings.cpu())

node_embeddings = torch.cat(all_embeddings, dim=0)
print(f"\n✓ Generated embeddings shape: {node_embeddings.shape}")

# Pad/truncate to target dimension
current_dim = node_embeddings.shape[1]
target_dim = CONFIG['embedding_dim']

if current_dim < target_dim:
    padding = torch.zeros(node_embeddings.shape[0], target_dim - current_dim)
    node_embeddings = torch.cat([node_embeddings, padding], dim=1)
    print(f"  Padded from {current_dim} to {target_dim} dimensions")
elif current_dim > target_dim:
    node_embeddings = node_embeddings[:, :target_dim]
    print(f"  Truncated from {current_dim} to {target_dim} dimensions")

print(f"\n✓ Final embeddings shape: {node_embeddings.shape}")

# Free GPU memory
del bert_model, tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()
    print(f"✓ Freed GPU memory")

## 6. Create Multi-hop Graph Embeddings

Aggregate node embeddings from k-hop neighborhoods.

In [ ]:
print("="*70)
print(" "*20 + "STEP 3: Creating Multi-hop Embeddings")
print("="*70)

num_nodes = data.num_nodes
embedding_dim = node_embeddings.shape[1]
num_hops = CONFIG['num_hops']
sample_size = CONFIG['sample_size']

# Initialize multi-hop embeddings
multi_hop_embs = torch.zeros(num_hops, num_nodes, embedding_dim)

print(f"Building {num_hops}-hop neighborhoods for {num_nodes:,} nodes...")
print(f"Sampling up to {sample_size} neighbors per hop\n")

for node_idx in tqdm(range(num_nodes), desc="Creating multi-hop embeddings"):
    for hop in range(num_hops):
        if hop == 0:
            # Hop 0: Use the node's own embedding
            multi_hop_embs[hop, node_idx] = node_embeddings[node_idx]
        else:
            # Get k-hop subgraph
            subset, _, _, _ = k_hop_subgraph(
                node_idx=node_idx,
                num_hops=hop,
                edge_index=data.edge_index,
                relabel_nodes=False,
                num_nodes=num_nodes
            )
            
            # Exclude the center node itself
            neighbors = [n for n in subset.tolist() if n != node_idx]
            
            if len(neighbors) > 0:
                # Sample neighbors if there are too many
                if len(neighbors) > sample_size:
                    sampled_neighbors = torch.tensor(
                        np.random.choice(neighbors, sample_size, replace=False)
                    )
                else:
                    sampled_neighbors = torch.tensor(neighbors)
                
                # Aggregate neighbor embeddings (mean pooling)
                neighbor_embs = node_embeddings[sampled_neighbors]
                multi_hop_embs[hop, node_idx] = neighbor_embs.mean(dim=0)
            else:
                # No neighbors at this hop: use the node's own embedding
                multi_hop_embs[hop, node_idx] = node_embeddings[node_idx]

print(f"\n✓ Multi-hop embeddings shape: {multi_hop_embs.shape}")
print(f"  Expected: [{num_hops}, {num_nodes}, {embedding_dim}]")
print(f"\nEmbedding structure:")
print(f"  - Hop 0: Node itself")
print(f"  - Hop 1-{num_hops-1}: Aggregated k-hop neighbors")

## 7. Split Data and Create Conversations

Create train/test split and format as conversations.

In [ ]:
print("="*70)
print(" "*20 + "STEP 4: Creating Train/Test Data")
print("="*70)

# Split data
print("Splitting into train/test sets...")
train_idx, test_idx = train_test_split(
    range(data.num_nodes),
    test_size=CONFIG['test_size'],
    random_state=CONFIG['random_state'],
    stratify=data.y.cpu().numpy()
)

print(f"✓ Split complete:")
print(f"  Training samples: {len(train_idx):,}")
print(f"  Test samples: {len(test_idx):,}")
print(f"  Split ratio: {1-CONFIG['test_size']:.0%} / {CONFIG['test_size']:.0%}")

# Create conversations
def create_conversations(indices, data):
    conversations = []
    for idx in tqdm(indices, desc="Creating conversations"):
        node_id = int(idx)
        label_idx = int(data.y[node_id].item())
        label_name = data.label_texts[label_idx]
        node_text = data.raw_texts[node_id]
        
        conversations.append({
            "id": [node_id],
            "conversations": [
                {
                    "from": "human",
                    "value": f"What category does this paper belong to? Paper: {node_text}"
                },
                {
                    "from": "gpt",
                    "value": label_name
                }
            ],
            "graph": 1
        })
    return conversations

print("\nCreating conversation data...")
train_conversations = create_conversations(train_idx, data)
test_conversations = create_conversations(test_idx, data)

print(f"\n✓ Created {len(train_conversations):,} train + {len(test_conversations):,} test conversations")

# Show class distribution
print("\nClass distribution (train):")
train_labels = [data.y[i].item() for i in train_idx]
for i, label in enumerate(data.label_texts):
    count = train_labels.count(i)
    print(f"  {label}: {count:,}")

## 8. Save All Files

Save conversations and embeddings to `/kaggle/working/`.

In [ ]:
print("="*70)
print(" "*20 + "STEP 5: Saving Files")
print("="*70)

output_dir = Path(CONFIG['output_dir'])
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {output_dir}\n")

# Save train conversations
train_path = output_dir / "cora_train_node_data.jsonl"
with open(train_path, 'w', encoding='utf-8') as f:
    for conv in train_conversations:
        f.write(json.dumps(conv) + '\n')
train_size = os.path.getsize(train_path) / (1024**2)
print(f"✓ Saved: {train_path.name}")
print(f"  Samples: {len(train_conversations):,}")
print(f"  Size: {train_size:.2f} MB")

# Save test conversations
test_path = output_dir / "cora_test_node_data.jsonl"
with open(test_path, 'w', encoding='utf-8') as f:
    for conv in test_conversations:
        f.write(json.dumps(conv) + '\n')
test_size = os.path.getsize(test_path) / (1024**2)
print(f"\n✓ Saved: {test_path.name}")
print(f"  Samples: {len(test_conversations):,}")
print(f"  Size: {test_size:.2f} MB")

# Save base node embeddings
node_emb_path = output_dir / "node_embeddings.pt"
torch.save(node_embeddings, node_emb_path)
node_emb_size = os.path.getsize(node_emb_path) / (1024**2)
print(f"\n✓ Saved: {node_emb_path.name}")
print(f"  Shape: {node_embeddings.shape}")
print(f"  Size: {node_emb_size:.2f} MB")

# Save multi-hop embeddings
multi_hop_path = output_dir / "multi_hop_graph_embedding.pt"
torch.save(multi_hop_embs, multi_hop_path)
multi_hop_size = os.path.getsize(multi_hop_path) / (1024**2)
print(f"\n✓ Saved: {multi_hop_path.name}")
print(f"  Shape: {multi_hop_embs.shape}")
print(f"  Size: {multi_hop_size:.2f} MB")

# Calculate total size
total_size = train_size + test_size + node_emb_size + multi_hop_size

print(f"\n{'='*70}")
print(f"✅ ALL FILES SAVED SUCCESSFULLY!")
print(f"{'='*70}")
print(f"Total size: {total_size:.2f} MB")
print(f"Location: {output_dir}")

## 9. Summary and Download Instructions

In [ ]:
print("="*70)
print(" "*20 + "✅ PREPARATION COMPLETE!")
print("="*70)

print("\n📊 Summary:")
print(f"  • Dataset: Cora Citation Network")
print(f"  • Total nodes: {data.num_nodes:,}")
print(f"  • Train samples: {len(train_conversations):,}")
print(f"  • Test samples: {len(test_conversations):,}")
print(f"  • Classes: {len(data.label_texts)}")
print(f"  • Multi-hop embeddings: {multi_hop_embs.shape}")
print(f"  • Total output size: {total_size:.2f} MB")

print("\n📥 Download Instructions:")
print("  1. Click 'Output' tab on the right sidebar")
print("  2. Click 'Download All' to get cora_gwm_data.zip")
print("  3. Or download files individually:")

for file in sorted(output_dir.glob("*")):
    size_mb = os.path.getsize(file) / (1024**2)
    print(f"     • {file.name} ({size_mb:.2f} MB)")

print("\n🚀 Next Steps:")
print("  1. Download the files from Kaggle")
print("  2. Upload to your training environment")
print("  3. Update file paths in GWM-E training script")
print("  4. Run training:")
print("     python train.py \\")
print("       --train_jsonl cora_train_node_data.jsonl \\")
print("       --test_jsonl cora_test_node_data.jsonl \\")
print("       --embedding_path multi_hop_graph_embedding.pt")

print("\n" + "="*70)

# Show example conversation
print("\n📝 Example Conversation:")
print("-"*70)
example = json.dumps(train_conversations[0], indent=2)
if len(example) > 600:
    print(example[:600] + "\n...")
else:
    print(example)

print("\n" + "="*70)
print("Notebook completed successfully! 🎉")
print("="*70)

## Optional: Verify Files

In [ ]:
# Quick verification
print("Verifying saved files...\n")

# Load and check embeddings
loaded_embs = torch.load(multi_hop_path)
print(f"✓ Multi-hop embeddings: {loaded_embs.shape}")
assert loaded_embs.shape == multi_hop_embs.shape

# Load and check conversations
with open(train_path, 'r') as f:
    train_lines = f.readlines()
print(f"✓ Train conversations: {len(train_lines)} lines")
assert len(train_lines) == len(train_conversations)

with open(test_path, 'r') as f:
    test_lines = f.readlines()
print(f"✓ Test conversations: {len(test_lines)} lines")
assert len(test_lines) == len(test_conversations)

print("\n✅ All files verified successfully!")